In [8]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [36]:
crypto_data = pd.read_csv("Resources/crypto_data.csv")
crypto_data.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [37]:
crypto_data.drop(columns="IsTrading", inplace = True)

In [38]:
crypto_data.dropna(inplace = True)

In [39]:
crypto_data.rename(columns={"Unnamed: 0" : "Index"}, inplace=True)
crypto_data = crypto_data.set_index("Index")

In [40]:
crypto_data.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Index,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [43]:
# Keep the rows where coins are mined.
crypto_data = crypto_data.loc[crypto_data["TotalCoinsMined"] != 0]
crypto_data.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Index,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [45]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names = crypto_data[["CoinName"]]
crypto_names.head()

,CoinName
Index,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [50]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
clust_algo = crypto_data.drop(columns="CoinName")
clust_algo.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Index,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [51]:
clust_algo = pd.get_dummies(clust_algo, columns=["Algorithm", "ProofType"])
clust_algo.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Index,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Standardize the data with StandardScaler().
std_array = StandardScaler().fit_transform(clust_algo)

In [53]:
std_array

array([[-0.11416167, -0.15072664, -0.04163054, ..., -0.04163054,
        -0.04163054, -0.04163054],
       [-0.09006124, -0.142553  , -0.04163054, ..., -0.04163054,
        -0.04163054, -0.04163054],
       [ 0.55458069,  4.67601177, -0.04163054, ..., -0.04163054,
        -0.04163054, -0.04163054],
       ...,
       [-0.11399502, -0.150404  , -0.04163054, ..., -0.04163054,
        -0.04163054, -0.04163054],
       [-0.11415874, -0.15071128, -0.04163054, ..., -0.04163054,
        -0.04163054, -0.04163054],
       [-0.11367081, -0.14919024, -0.04163054, ..., -0.04163054,
        -0.04163054, -0.04163054]])

In [80]:
# Initialize PCA model
pca = PCA(n_components=3)


In [81]:
# Using PCA to reduce dimension to three principal components.
crypto_pca = pca.fit_transform(std_array)
crypto_pca

array([[-0.32298296,  1.14693518, -0.39723828],
       [-0.3056183 ,  1.14718158, -0.39731703],
       [ 2.42121667,  1.74994786, -0.45408084],
       ...,
       [-0.15774821, -2.10733921,  0.13090294],
       [-0.27578084,  0.78192053, -0.22226319],
       [-0.32197246,  1.14689898, -0.39724446]])

In [82]:
# Create a DataFrame with the three principal components.
crypto_pca_df = pd.DataFrame(data=crypto_pca, index = clust_algo.index, columns=["PC 1", "PC 2", "PC 3"])
crypto_pca_df.head()

,PC 1,PC 2,PC 3
Index,,,
42,-0.322983,1.146935,-0.397238
404,-0.305618,1.147182,-0.397317
1337,2.421217,1.749948,-0.454081
BTC,-0.139223,-1.333985,0.148047
ETH,-0.136645,-2.067827,0.234586


In [93]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)

In [95]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [104]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(crypto_pca)
# Predict clusters
predictions = model.predict(crypto_pca)
predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,

In [105]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_data.join(crypto_pca_df, on=crypto_data.index, how='outer').drop(columns="key_0")

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
clustered_df.head(20)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class
Index,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.322983,1.146935,-0.397238,1
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.305618,1.147182,-0.397317,1
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.421217,1.749948,-0.454081,1
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.139223,-1.333985,0.148047,0
ETH,Ethereum,Ethash,PoW,1.076842e+08,0,-0.136645,-2.067827,0.234586,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000,-0.158015,-1.066375,-0.028552,0
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000,-0.380503,1.290229,-0.294675,1
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0,-0.146952,-2.332572,0.283130,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000,-0.135025,-2.067922,0.234574,0


In [128]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name = "CoinName",
    hover_data = ['Algorithm'],
    width=800
)

fig.update_layout(legend=dict(x=0,y=1))

fig.show()

In [129]:
import hvplot.pandas

clustered_df.hvplot.table()

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class]

In [132]:
# Print the total number of tradable cryptocurrencies.
print("There are", len(clustered_df), "tradable cryptocurrencies.")

There are 578 tradable cryptocurrencies.


In [134]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler()

scaled = scaler.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])

scaled

array([[4.20000000e-11, 5.94230127e-03],
       [5.32000000e-04, 7.00182308e-03],
       [3.14159265e-01, 3.53420682e-02],
       ...,
       [2.10000000e-05, 5.94962775e-03],
       [1.00000000e-06, 5.94243008e-03],
       [1.00000000e-04, 5.96388077e-03]])

In [138]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_df = pd.DataFrame(scaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index = clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df["Class"] = clustered_df["Class"]

scaled_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
Index,,,,
42,4.200000e-11,0.005942,42 Coin,1
404,5.320000e-04,0.007002,404Coin,1
1337,3.141593e-01,0.035342,EliteCoin,1
BTC,2.100000e-05,0.005960,Bitcoin,0
ETH,0.000000e+00,0.006050,Ethereum,0
LTC,8.400000e-05,0.006006,Litecoin,0
DASH,2.200000e-05,0.005951,Dash,1
XMR,0.000000e+00,0.005960,Monero,0
ETC,2.100000e-04,0.006056,Ethereum Classic,0


In [141]:
scaled_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply")

:Scatter   [TotalCoinsMined]   (TotalCoinSupply)